In [ ]:
# import json

# with open('bot_tmp.log', encoding='utf-8') as f:
#     lines = [line.split(" --- ") for line in f.readlines()]

#     res = {'files': []}

#     for idx, line in enumerate(lines):
#         entry = {
#             'id': idx + 1,
#             'type': line[1].strip()[1:-1],
#             'name': line[0].strip()[1:-1],
#             'uploaded_date': line[2].strip()[1:-1]
#         }

#         res['files'].append(entry)

#     with open('files_info.json', 'w', encoding='utf-8') as f:
#         json.dump(res, f, indent=4, ensure_ascii=False)

In [ ]:
# WARNING -> This doesn't work reliable.... It deletes data that are matched!!!!
# import json

# auxiliaryList = []

# with open('../files_info.json', 'r+', encoding='utf-8') as f:
#     data = json.load(f)

#     for info in data["files"]:
#         info["id"] = 1

#     for idx, info in enumerate(data["files"]):
#         if info not in auxiliaryList:
#             auxiliaryList.append(info)

#     for idx, info in enumerate(auxiliaryList):
#         info["id"] = idx + 1

#     print(len(data['files']))
    
# with open('../files_info.json', 'w', encoding='utf-8') as f:
#     data["files"] = auxiliaryList
#     json.dump(data, f, indent=4, ensure_ascii=False)

In [ ]:
import os
import time
import glob
import json
import string
import platform
import unicodedata
from selenium import webdriver
from json.decoder import JSONDecodeError
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# Packages & Libraries
import json
from json.decoder import JSONDecodeError
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC

from constants import (
    secrets,
    encoding,
    target_file_type,
    log_file_location,
    normalization_form,
    explicit_wait_time,
    tracker_file_location,
    network_failure_timeout,
)

from helperFunctions import (
    login,
    updateLog,
    waitNSeconds,
    downloadFile,
    loadMoreFiles,
    appendFilesInfo,
    initializeWebpage,
    checkDownloadStatus,
    initializeWebDriver,
    waitToFinishDownload,
    getExistingFilesInfo,
)

"""
 targets identifiers -> this identifiers will change continuously so update it according to your needs 

"""
download_button_cssSelector = "a[href*='https://www.facebook.com/download/']"
permalink_xpath = "//a[contains(@href, 'https://www.facebook.com/groups/201623576939858/permalink/')]"  # by default finds 15 per-scroll
fileOption_xpath = "//div[@aria-label='File options']"  # by default finds 15 per-scroll

# Target identifiers
# They change too often so they have to be present here
fileName_xpath = "//span[@class='d2edcug0 hpfvmrgz qv66sw1b c1et5uql lr9zc1uh a8c37x1j fe6kdd0r mau55g9w c8b282yb keod5gw0 nxhoafnm aigsh9s9 d3f4x2em iv3no6db jq4qci2q a3bd9o3v lrazzd5p oo9gr5id hzawbc8m']"  # find 15 per-scroll
fileTypeDate_xpath = "//span[@class='d2edcug0 hpfvmrgz qv66sw1b c1et5uql lr9zc1uh a8c37x1j fe6kdd0r mau55g9w c8b282yb keod5gw0 nxhoafnm aigsh9s9 d9wwppkn iv3no6db e9vueds3 j5wam9gi b1v8xokw oo9gr5id hzawbc8m']"  # find 30 per-scroll

driver, wait = initializeWebDriver()
login(driver, wait)

In [ ]:
def compareString(s1, s2):
    # string will be normalized before coming here
    # s1 = unicodedata.normalize(normalization_form, s1)
    # s2 = unicodedata.normalize(normalization_form, s2)

    # Removing whitespace in the string before comparing
    # because when file saves in the machine it seems to add whitepsaces after '-'
    # remove = string.punctuation + string.whitespace
    remove = string.whitespace
    mapping = {ord(c): None for c in remove}

    return s1.translate(mapping) == s2.translate(mapping)


def getExistingFilesInfo():
    with open(tracker_file_location, "r", encoding=encoding) as f:
        try:
            files_info = json.load(f)["files"]
        except JSONDecodeError:
            files_info = []

    tracked_files = sorted(
        [
            [
                unicodedata.normalize(normalization_form, info["uploaded_date"]),
                unicodedata.normalize(normalization_form, info["name"]),
            ]
            for info in files_info
        ]
    )

    return [], tracked_files


def binarySearch(item, itemList, multipleCheck=False):
    left = 0
    right = len(itemList) - 1

    while left <= right:
        mid = left + (right - left) // 2

        if multipleCheck:
            """
            Sorted By: uploaded date

            If permalink exist for the file then check:
                0 uploaded date
                1 permalink of the post
                2 file name
            If not then check:
                0 uploaded date
                2 file name

            """
            if compareString(itemList[mid][0], item[0]) and compareString(
                itemList[mid][1], item[1]
            ):
                return mid
            elif itemList[mid][0] > item[0]:
                right = mid - 1
            else:
                left = mid + 1
        else:
            if compareString(itemList[mid], item):
                return mid
            elif itemList[mid] > item:
                right = mid - 1
            else:
                left = mid + 1

    return -1


In [ ]:
j, k = 0, 1

# Grabbing initial loadeded target
try:
    file_option_buttons = wait.until(
        EC.presence_of_all_elements_located((By.XPATH, fileOption_xpath))
    )
    file_names = wait.until(
        EC.presence_of_all_elements_located((By.XPATH, fileName_xpath))
    )
    file_types_and_dates = wait.until(
        EC.presence_of_all_elements_located((By.XPATH, fileTypeDate_xpath))
    )

    # This is the unique identifier of a post/file
    post_permalink = wait.until(
        EC.presence_of_all_elements_located((By.XPATH, permalink_xpath))
    )
except TimeoutException:
    print("XPATH's identifier (class names) has changed again! 🤯")


""" MAIN LOOP """
for idx, button in enumerate(file_option_buttons):
    try:
        _name = file_names[idx].text
        _type = file_types_and_dates[j].text
        _date = file_types_and_dates[k].text
        _permalink = post_permalink[idx].get_attribute("href")

        # as 'file_types_and_dates' contains both file-type and date
        j += 2
        k += 2

        # Scrolling after it reaches at the end of the list to load more files
        if button is file_option_buttons[-1]:
            [
                file_option_buttons,
                file_names,
                file_types_and_dates,
                post_permalink,
            ] = loadMoreFiles(
                driver,
                [file_option_buttons, file_names, file_types_and_dates, post_permalink],
                [fileOption_xpath, fileName_xpath, fileTypeDate_xpath, permalink_xpath],
            )

            updateLog("\nTotal Loaded Files: {}\n".format(len(file_option_buttons)))

        """
        Download pdf files only
        Cannot check this before the scroll because
        here is a possibility to have more than one page worth of non-pdf files
        
        """
        if not _type == target_file_type.upper():
            updateLog(
                '\n😪 Skipping ({}): "{} --- {}", 🤔 Reason: FILE_TYPE: "{}"'.format(
                    idx + 1, _name, _date, _type
                )
            )
            continue

        

    except Exception as e:
        updateLog("\n*** ERROR at {}, date: {} ***\n--> {}\n".format(_name, _date, e))
